In [1]:
import warnings
import re

from scripts.train import train
from dataloading.metadata_generator import generate_metadata

%load_ext autoreload
%autoreload 2

# ignore certain warnings
warnings.filterwarnings(
    "ignore", 
    message="The video decoding and encoding capabilities of torchvision are deprecated from version 0.22"
)

/home/ben/Thesis/ForgeryLocalization/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Goal: Run training for BA-TFD on a small subset of AV-Deepfake1M-PlusPlus to test the setup.

Args:

In [ ]:
args = {
    "config": 'models/batfd/batfd.toml',
    "data_root": 'data/',
    "batch_size": 8,
    "num_workers": 11,
    "gpus": 1,
    "precision": 32,
    "num_train": 100,
    "num_val": 100,
    "max_epochs": 1,
    "logger": "tensorboard",
    "resume": None
}

In [3]:
# prepare metadata
generate_metadata(args["data_root"])

Existing metadata file data/train_metadata.json has been removed.
Metadata for train saved to data/train_metadata.json
Existing metadata file data/val_metadata.json has been removed.
Metadata for val saved to data/val_metadata.json
Existing file list data/testA_files.txt has been removed.
File list for testA saved to data/testA_files.txt


In [ ]:
train(args)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model initialized!
DataModule initialized!
Trainer initialized!
Load 2608 data in train.
Load 735 data in val.
Load 1419 data in testA.



  | Name                   | Type                              | Params | Mode 
-------------------------------------------------------------------------------------
0 | video_encoder          | C3DVideoEncoder                   | 2.3 M  | train
1 | audio_encoder          | CNNAudioEncoder                   | 76.4 K | train
2 | video_frame_classifier | FrameLogisticRegression           | 257    | train
3 | audio_frame_classifier | FrameLogisticRegression           | 257    | train
4 | video_boundary_module  | BoundaryModule                    | 1.5 M  | train
5 | audio_boundary_module  | BoundaryModule                    | 1.5 M  | train
6 | fusion                 | ModalFeatureAttnBoundaryMapFusion | 32.8 K | train
7 | frame_loss             | BCEWithLogitsLoss                 | 0      | train
8 | contrast_loss          | ContrastLoss                      | 0      | train
9 | bm_loss                | MSELoss                           | 0      | train
---------------------------------

Epoch 0:  19%|█▉        | 24/125 [01:25<06:01,  0.28it/s, v_num=13]        